## Podsumowanie poprzednich zajęć 

Na poprzednich dwóch zajęciach laboratoryjnych zrealizowaliśmy zagadnienia dotyczące przetwarzania danych ustrukturyzowanych i nieustrukturyzowanych w trybie wsadowym. Ponadto przygotowaliśmy środowisko produkcyjne (z wykorzystaniem biblioteki FLASK) wykorzystujące model napisany w pełni obiektowo otrzymany po wstępnym przetworzeniu danych (irys).   

1. Ustrukturyzowane dane - tablice numpy, ramki danych Pandas, tabele danych w bazach SQL (tworzenie, filtrowanie, modyfikacja)
2. Nieustrukturyzowane dane - JSON, tensory numpy (odczyt, zapis, przetworzenie)
3. Wykorzystanie obiektowego programowania w Pythonie - podstawa budowy klas, tworzenie obiektów, korzystanie z pól obiektów i metod (funkcji). 
4. Stworzenie modelu klasyfikacji binarnej opartego o sieć perceprtornu oraz wykorzystującą algorytm Adeline napisany obiektowo w pełnej analogii do modeli z biblioteki sklearn.
5. Wykorzystanie środowiska SQLAlchemy do łączenia się z bazami danych. 
6. Strona www realizująca API z wykorzystaniem modelu - nowe dane w czasie rzeczywistym + prognoza - Jako system odpytywania modelu w czasie rzeczywistym (Zastanów się jak go unowocześnić) 


Podczas przerabiania dowolnych technik uczenia maszynowego najczęściej (jeśli nie zawsze) jesteśmy uczeni realizacji zadań takiego systemu z podziałem na trzy podstawowe kategorie:

1. Uczenie nadzorowane  - supervised learning
    - klasyfikacja - zrealizowany na poprzednich ćwiczeniach
    - Regresja liniowa
2. Uczenie nienadzorowane - unsupervised learning
3. Uczenie przez wzmacnianie - reinforcement learning

Jednak systemy te można również klasyfikować ze względu na `możliwość trenowania przyrostowego przy użyciu strumienia nadsyłanych danych`

1. **Uczenie wsadowe - batch learning**. To system w którym do jego nauki musisz wykorzysać wszytkie zapisane i już istniejące dane. Zajmuje zazwyczaj dużo czasu i zasobów - przeprowadzany w trybie offline. System wpierw jest uczony, a następnie zostaje wdrożony do cyklu produkcyjnego i już więcej nie jest trenowany (korzysta tylko ze zdobytych wcześniej informacji). Zajwisko to nazywane jest **uczeniem offline**. 

Jeśli chcesz aby system uczenia wsadowego brał pod uwagę nowe dane to musisz od podstaw wytrenować nową wersję systemu przy użyciu wszystkich dostępnch danych, wyłączyć stary system i zastąpić go nowym. Na szczęście proces ten jest w pełni automatyzowalny. Jednak trzeba pamiętać, iż trenowanie nowego modelu na pełnym zbiorze danych może trwać bardzo długo (i jest dość kosztowne) stąd wymiana modeli pojawia się np raz na tydzień raz na dzień. W przypadku bardzo dużej ilości informacji system taki może szybko przestać działać - zamiast wykonywać swoje zadania będzie obliczał nowy model. 

2. W procesie **uczenia przyrostowego - online learning** system trenowany jest na bieżąco poprzez sekwencyjne dostarczanie danych (pojedyncze lub minipaczki - mini-batches). Każdy krok uczenia jest szybki i mało kosztowny. Uczenie następuje w momencie pojawienia się nowych danych.  

Uczenie przyrostowe sprawdza się wszędzie tam gdzie układ odbiera ciągły strumień danych (urządzenia IoT, giełda) i wymagana jest szybkie i autonomiczne dopasowanie do nowych warunków. Przydaje się również przy pracy z ograniczonymi zasobami obliczeniowymi (stare dane nie są istotne).

Dużym problemem uczenia przyrostowego jest stopniowy spadek wydajności systemu w przypadku gdy dostarczone dane przestają być prawidłowe. Np. uszkodzony czujnik, celowe zasypywanie przeglądarki danymi w celu podbicia rankingu w wynikach wyszukiwania (algorytmy wykrywania anomalii).


[Stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

[Stochastic learning](https://leon.bottou.org/publications/pdf/mlss-2003.pdf)

## Środowisko Apache SPARK

[książka](https://pages.databricks.com/rs/094-YMS-629/images/LearningSpark2.0.pdf) 


1. Silnik analityczny do przetwarzania danych na dużą skalę
2. Projekt open source, od 2013 w Apache Software FOundation
3. Napisany w Scali 
4. Udostępnia API w Java, Scala, Python, R 


### Instalacja i uruchomienie 

1. Wersja trywialna (Docker) 

```{bash}
docker run -d -p 8888:8888 -v "full_path_to_your_folder:/notebooks" sebkaz/docker-spark-jupyter
```

```{bash}
docker run -d -p 8888:8888 -v "full_path_to_your_folder:/notebooks" jupyter/pyspark-notebook
```

2. Wersja trywialna trywialna (komputer ze środowiskiem Python + JDK JAVA przynajmniej w wersji 8) 

    - [Ściągnij katalog](https://www.apache.org/dyn/closer.lua/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz)
    - Rozpakuj np 7z
    - umieść w wygodnym miejscu i zapisz ścieżkę (będzie potrzebna do findspark() )
    - uruchom jupyter notebook'a


In [ ]:
import findspark
# findspark.init()
# findspark.init("C:/Users/SebastianZajac/Desktop/spark")
findspark.init("/Users/air/Desktop/spark/") # on my mac

### SparkContext

1. Główny, podstawowy obiekt
2. Punkt wejścia do pracy ze Sparkiem
3. Generowanie obiektów RDD

In [ ]:
# inicjalizacja SparkContext
from pyspark import SparkContext
sc = SparkContext(appName="myAppName")

In [ ]:
sc

### SparkSession

1. Główny punkt wyjścia do SparkSQL
2. Opakowuje (wrapper) SparkContext
3. Zazwyczaj pierwszy obiekt, który będziemy tworzyć

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("new")\
        .getOrCreate()
# otrzymanie obiektu SparkContext
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
sc

### RDD

- Resilient Distributed Dataset
- Podstawowa abstrakcja oraz rdzeń Sparka
- Obsługiwane przez dwa rodzaje operacji:
    - Akcje:
        - operacje uruchamiająceegzekucję transformacji na RDD
        - przyjmują RDD jako input i zwracają wynik NIE będący RDD
    - Transformacje:
        - leniwe operacje
        - przyjmują RDD i zwracają RDD

- In-Memory - dane RDD przechowywane w pamięci
- Immutable 
- Lazy evaluated
- Parallel - przetwarzane równolegle
- Partitioned - rozproszone 

## WAŻNE informacje !

Ważne do zrozumienia działania SPARKA:

Term                   |Definition
----                   |-------
RDD                    |Resilient Distributed Dataset
Transformation         |Spark operation that produces an RDD
Action                 |Spark operation that produces a local object
Spark Job              |Sequence of transformations on data with a final action


Dwie podstawowe metody tworzenia RDD:

Method                      |Result
----------                               |-------
`sc.parallelize(array)`                  |Create RDD of elements of array (or list)
`sc.textFile(path/to/file)`                      |Create RDD of lines from file

Podstawowe transformacje

Transformation Example                          |Result
----------                               |-------
`filter(lambda x: x % 2 == 0)`           |Discard non-even elements
`map(lambda x: x * 2)`                   |Multiply each RDD element by `2`
`map(lambda x: x.split())`               |Split each string into words
`flatMap(lambda x: x.split())`           |Split each string into words and flatten sequence
`sample(withReplacement=True,0.25)`      |Create sample of 25% of elements with replacement
`union(rdd)`                             |Append `rdd` to existing RDD
`distinct()`                             |Remove duplicates in RDD
`sortBy(lambda x: x, ascending=False)`   |Sort elements in descending order

Podstawowe akcje 

Action                             |Result
----------                             |-------
`collect()`                            |Convert RDD to in-memory list 
`take(3)`                              |First 3 elements of RDD 
`top(3)`                               |Top 3 elements of RDD
`takeSample(withReplacement=True,3)`   |Create sample of 3 elements with replacement
`sum()`                                |Find element sum (assumes numeric elements)
`mean()`                               |Find element mean (assumes numeric elements)
`stdev()`                              |Find element deviation (assumes numeric elements)

-----
- **parallelize(c)** - tworzenie RDD na podstawie lokalnej kolekcji
- **map(f)** - zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD (**T**)
- **filter(f)** - zwraca nowe RDD zawierające jedynie elementy które spełniają predykat (**T**)
- **reduce(f)** - agreguje elementy zbioru wykorzystując podaną funkcję. Funkcja redukująca musi być asocjacyjna [(a x b) x c = a x (b x c)] i przemienna [a x b = b x a] (**A**)

In [ ]:
rdd = sc.parallelize(range(10)) # utworzenie RDD 

rdd

In [ ]:
rdd.collect() # akcja 

In [ ]:
rdd.take(2)

In [ ]:
rdd.takeSample(True,3)

In [ ]:
rdd.takeSample(False,3)

In [ ]:
rdd.count()

In [ ]:
rdd.mean()

In [ ]:
%%file example.txt
first 
second line
the third line
then a fourth line

In [ ]:
text_rdd = sc.textFile('example.txt')

In [ ]:
text_rdd.first()

In [ ]:
text_rdd.take(3)

In [ ]:
text_rdd.takeSample(True,2)

In [ ]:
text_rdd.takeSample(False,2)

In [ ]:
text_rdd.count()

In [ ]:
rdd1 = sc.parallelize(range(1,20))
rdd2 = sc.parallelize(range(10,25))
rdd3 = rdd1.union(rdd2)
rdd3.collect()

In [ ]:
rdd4 = rdd3.distinct()
rdd4.collect()

-----
- **parallelize(c)** - tworzenie RDD na podstawie lokalnej kolekcji
- **map(f)** - zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD (**T**)
- **filter(f)** - zwraca nowe RDD zawierające jedynie elementy które spełniają predykat (**T**)
- **reduce(f)** - agreguje elementy zbioru wykorzystując podaną funkcję. Funkcja redukująca musi być asocjacyjna [(a x b) x c = a x (b x c)] i przemienna [a x b = b x a] (**A**)

#### Map vs. FlatMap

In [ ]:
text_rdd.map(lambda line: line.split()).collect()

In [ ]:
# Collect everything as a single flat map
text_rdd.flatMap(lambda line: line.split()).collect()

In [ ]:
sc.parallelize(range(20)) \
.map(lambda x: x * 2) \
.filter(lambda x: x != 2) \
.reduce(lambda x,y: x + y)

In [ ]:
sc.parallelize(range(20)).collect() # rozproszenie -> parallelize -> collect powrot

In [ ]:
sc.parallelize(range(20)).map(lambda x: x * 2).collect()

In [ ]:
sc.parallelize(range(20)) \
.map(lambda x: x * 2).filter(lambda x: x != 2).collect()

----
- **textFile(p)** - tworzenie RDD na podstawie pliku. Jeden wiersz = jeden element RDD
- **flatMap(f)** - zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD oraz spłaszczeniu rezultatu (**T**)
- **reduceByKey(f)** - zwraca RDD z połączonymi wartościami dla każdego klucza. Funkcja redukująca musi być asocjacyjna [(a x b) x c = a x (b x c)] i przemienna [a x b = b x a] (**T**)
- **collect()** - zwraca elementy RDD na driver (**A**)

In [ ]:
import re

In [ ]:
sc.textFile("RDD_input") \
.map(lambda x: re.findall(r"[a-z']+", x.lower())) \
.flatMap(lambda x: [(y, 1) for y in x]) \
.reduceByKey(lambda x,y: x + y) \
.collect()

In [ ]:
sc.textFile("RDD_input") \
.map(lambda x: re.findall(r"[a-z']+", x.lower())) \
.flatMap(lambda x: [(y, 1) for y in x]).collect()

In [ ]:
sc.textFile("RDD_input") \
.map(lambda x: re.findall(r"[a-z']+", x.lower())).collect()

> ZADANIE

In [ ]:
rawMD = sc.textFile("MobyDick.txt")
rawMD.take(10)

### SQL 

In [ ]:
empHist = spark.read.parquet("salary_hist")


In [ ]:
empHist.show()

In [ ]:
empHist.select("name").explain()

In [ ]:
adultDF = spark.read.csv("adult.data", inferSchema=True, ignoreLeadingWhiteSpace=True)

In [ ]:
adultDF.take(1)

In [ ]:
col_names = ["age", "workclass", "fnlwgt", "education", "education-num","marital-status", "occupation", 
             "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", 
             "native-country", "earnings"]

In [ ]:
adultDF = adultDF.toDF(*col_names).drop("fnlwgt").dropna("any")

In [ ]:
adultDF.show(3, vertical=True)

In [ ]:
adultDF.printSchema()

In [ ]:
adultDF.show(3)

In [ ]:
adultDF.groupBy('education').mean().show()

In [ ]:
adultDF.write.saveAsTable("adult_bucket")

In [ ]:
newAdult = spark.sql("select age, education, sex from adult_bucket where age > 50")

In [ ]:
newAdult.show(3)